In [379]:
# Importing libraries and dataframe
import pandas as pd
import numpy as np
df = pd.read_csv("datasets/games.csv")


In [380]:
# Setting columns to lowercase
df.columns = [column.lower() for column in df.columns]


In [381]:
#Look for duplicates, incorrect dtypes and nulls.
df.info()
print("_____________________")
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
_____________________


Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

 Name, Year_of_Release, Genre, Critic_Score, User_Score, Rating have missing values.

In [382]:
# I proceed removing duplicated rows.
df.duplicated().sum() 

0

Not duplicated rows found

In [383]:
# Identifying null values
df.isnull().sum()

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         6701
rating             6766
dtype: int64

In [384]:
df[df["name"].isnull()]
df[df["year_of_release"].isnull()]
df[df["genre"].isnull()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


- name column = There are 2 missing games, I'll proceed to delete both rows (not enough or relevant information to keep).
- year_of_release column = It's possible to look at the internet for the name of the game, the platform and get the year_of_release, I'll keep it for the moment. (I can look for an API)
- genre = Same missing values as name column, I'll delete it
- critic_score, user_score, rating = Can keep it like this because I don't have the source where they get the data.

In [385]:
# Checking before transforming to float
df["user_score"].unique()
df["user_score"].replace("tbd",np.nan, inplace=True)
#df["user_score"].replace(np.nan,-999, inplace=True) #If it is necessary

I decided to keep as an object for the moment

In [386]:
# Checking before transforming to float
df["critic_score"].unique()
df["critic_score"].replace(np.nan,-999, inplace=True)
df["critic_score"]= df["critic_score"].astype(int)

In [387]:
df["critic_score"].unique()

array([  76, -999,   82,   80,   89,   58,   87,   91,   61,   97,   95,
         77,   88,   83,   94,   93,   85,   86,   98,   96,   90,   84,
         73,   74,   78,   92,   71,   72,   68,   62,   49,   67,   81,
         66,   56,   79,   70,   59,   64,   75,   60,   63,   69,   50,
         25,   42,   44,   55,   48,   57,   29,   47,   65,   54,   20,
         53,   37,   38,   33,   52,   30,   32,   43,   45,   51,   40,
         46,   39,   34,   35,   41,   36,   28,   31,   27,   26,   19,
         23,   24,   21,   17,   22,   13])

In [388]:
df["rating"].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

We will keep the NaN values.

In [389]:
'''As an extra effort, I will conect to the IGDB API (https://www.igdb.com/api) to get the missing years. 
Should be necessary to extract the missing values from this list to use it as parameters (game_name and platform)
'''
games_missing_year_df = df[df["year_of_release"].isna()]
filtered_df = games_missing_year_df[["name","platform"]]

#I manually created this relation table to create a search parameter in the IGDB site.
filtered_df["platform"].unique() #getting the missing paltforms
relation_df = pd.read_csv("datasets/platform_relation.csv") 
fusion = pd.merge(left= filtered_df, right= relation_df, on="platform", how="left")


In [390]:
"The dataframe will be exported into a csv file to create a function which will search all the games by platform"

def export_to_csv(dataframe):
    '''Save DataFrame in a csv file using with'''
    with open("./datasets/filtered_games_db.csv", 'a', encoding="utf-8", newline="") as f:
        # f.tell() specifies if the file has been executed before
        empty = f.tell() == 0
        # If new, use the headers
        if empty:
            fusion.to_csv(f, index=False, header=True,)
        else:
            # If not, ignore the headers
            fusion.to_csv(f, index=False, header=False)

export_to_csv(fusion)

In [391]:
#This is the info requested and received by the API
processed_df = pd.read_csv("datasets/processed_game_list.csv")

#35 Cases where not identified by the API I did a manual research and save the file into a csv
manually_fill_df = pd.read_csv("datasets/games_not_found.csv")

#Homologating db to concat with the processed_df
manually_fill_df.rename(columns={"game name":"db_name"},inplace=True)
manually_fill_df

processed_df = pd.concat([processed_df, manually_fill_df], ignore_index=True)

In [392]:
#Mergin the dabase with the API_database
merged_df = pd.merge(games_missing_year_df, processed_df, left_on=["name","platform"], right_on=["db_name","platform"], how="left")
merged_df["year_of_release"] = merged_df["year"]
d_columns = ["release_date", "game_name", "db_name", "year"]
merged_df.drop(columns=d_columns, inplace=True)
merged_df.head()
# merged_df[merged_df["year_of_release"].isna()] #Majesty 2: The Fantasy Kingdom Sim was never released (cancelled)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Madden NFL 2004,PS2,2003.0,Sports,4.26,0.26,0.01,0.71,94,8.5,E
1,FIFA Soccer 2004,PS2,2003.0,Sports,0.59,2.36,0.04,0.51,84,6.4,E
2,LEGO Batman: The Videogame,Wii,2008.0,Action,1.80,0.97,0.00,0.29,74,7.9,E10+
3,wwe Smackdown vs. Raw 2006,PS2,2005.0,Fighting,1.57,1.02,0.00,0.41,-999,NaN,NaN
4,Space Invaders,2600,1978.0,Shooter,2.36,0.14,0.00,0.03,-999,NaN,NaN


In [394]:
#Passing the values to the current database with the original db
merged_df.shape #269 rows
df.loc[df['year_of_release'].isna(), 'year_of_release'] = merged_df['year_of_release'].values

In [397]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16714 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     16715 non-null  int32  
 9   user_score       7590 non-null   object 
 10  rating           9949 non-null   object 
dtypes: float64(5), int32(1), object(5)
memory usage: 1.3+ MB


In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             269 non-null    object 
 1   platform         269 non-null    object 
 2   year_of_release  268 non-null    float64
 3   genre            269 non-null    object 
 4   na_sales         269 non-null    float64
 5   eu_sales         269 non-null    float64
 6   jp_sales         269 non-null    float64
 7   other_sales      269 non-null    float64
 8   critic_score     269 non-null    int32  
 9   user_score       127 non-null    object 
 10  rating           181 non-null    object 
dtypes: float64(5), int32(1), object(5)
memory usage: 22.2+ KB
